In [6]:
import numpy as np
import random
from typing import Dict, List, Set
import logging

import seaborn as sns

# Step for ACO + Set Cover Code

In [7]:
""""Data process"""

class DataReader:


    def __init__(self, file):
        """
        DataReader Initialization
        
        Args:
            file (str): filepath 
        """
        self.file = file
        self.data = self._readTXT()
        self.universe, self.sets = tuple( int(i) for i in self.data[0].split() ) 
        self.sets, self.costs  = self._processTXT(self.data)

    def _readTXT(self):
        """
        Reads filepath.
        """
        with open(self.file) as f:
            data = f.readlines()
        return data

    def _processTXT(self, data):
        """
        Generate sets and costs
        
        Args:
            data (List): List of lines of data read
        
        """
        
        sets = [ [ int(j) for j in i.split()[1: ] ] for i in data[1:] ]
        costs = [ int(i.split()[0])  for i in data[1:] ]
        return  sets, costs


# Set Covering class

In [8]:
import logging


class BaseLogger:
    
    
    def __init__(self, logger_name: str = "Model Logger") -> None:
        
        self.logger_name = logger_name
        self.logger      =  logging.getLogger(self.logger_name)
        self.__logger_settings()
        # TODO: Check if the only handler needed is the StreamHandler
        if len(self.logger.handlers) > 2:
            for handler in self.logger.handlers: self.logger.removeHandler(handler)
        
        
    def __logger_settings(self) -> None:
        
        self.logger.setLevel(logging.INFO)
        logFileHandler = logging.FileHandler("./app.log",mode='w')
        consoleHandler  = logging.StreamHandler()
        logFormatter = logging.Formatter('%(name)s - %(levelname)s - [%(asctime)s] - %(message)s')
        
        consoleHandler.setFormatter(logFormatter)
        logFileHandler.setFormatter(logFormatter)
        
        self.logger.addHandler(logFileHandler)
        self.logger.addHandler(consoleHandler)


In [9]:
from collections import Counter

class SetCovering(BaseLogger):
    
    
    def __init__(self, subsets: List, costs: List) -> None:
        """ Set convering initialization
        """
        super().__init__()
        self.subsets = subsets
        self.costs   = costs
        self.universe = self.__identify_unique_items()
        self.total_set_elements = len(self.universe)
        self.total_subsets      = len(subsets)
        self.item_scores        = self.__calculate_item_scores()
        self.set_scores         = self.__calculate_set_scores()
        self.set_probabilities  = self.__calculate_set_probabilities()
        self.max_len_set        = max([len(subset) for subset in self.subsets])
    
    def __identify_unique_items(self) -> Set:
        """Find all unique elements of data structure
        
        """
        return { item for instance in self.subsets for item in instance }
    
    def __calculate_item_scores(self):
        item_counts  = dict(
            Counter([item for sublist in self.subsets for item in sublist])
        )
        
        item_values = {}
        for key, value in item_counts.items():
            item_values[key] = 1 / (value / self.total_subsets)
        
        SCORE_MAX = max(item_values.values())
        SCORE_MIN = min(item_values.values())
        
        item_scores = {}
        for key, value in item_values.items():
            item_scores[key] = SetCovering.max_min_normalizer(value, max_val = SCORE_MAX, min_val = SCORE_MIN)
            
        return item_scores

    def __calculate_set_probabilities(self):
        """Return the average score for each subset"""
        scores = [np.mean([self.item_scores[i] for i in subset])   for subset in self.subsets]
        
        return  self.__calculate_probabilities(scores)
    
    
    def __calculate_probabilities(self, vals):
        total_sum     = sum(vals)
        return [ val / total_sum  for val in vals ]
    
    def __calculate_set_scores(self):
        
        total_subsets = []
        for subset in self.subsets:
            subset_scores = [self.item_scores[i] for i in subset]
            subset_overall_score = np.average(subset_scores, weights= [item / sum(subset_scores) for item in subset_scores])
            total_subsets.append(subset_overall_score)
        return total_subsets
        
    
    def cover(self, probs = None):
        """ Create a set covering solution
        """
        
        if not probs:
            prob_dist = self.set_probabilities
        
        all_available_subsets = [*range(self.total_subsets)]
        
    
        covered = set()
        selected_subsets = []
        cost = 0
        while covered != self.universe:

            subset_idx = self.__select_set(set_list = all_available_subsets)
            subset     = set( self.subsets[subset_idx] )
            
            selected_subsets.append(subset_idx)
            covered |= subset
            
            cost += self.costs[subset_idx]
            all_available_subsets.remove(subset_idx)
            
        
        self.logger.info(f">>> Total covering cost: {cost}")
        self.logger.info(f">>> Total subsets selected: {len(selected_subsets)}")
        
        return selected_subsets, cost

        
    def __select_set(self, set_list: List, probs = None):
        """Select a set index, regarding if the set already have appended to a subset_list"""
        subset_idx = random.randint(0, len(set_list) - 1)
        return set_list[subset_idx]
    
    @staticmethod
    def max_min_normalizer(num, max_val, min_val):
        return (num - min_val) / (max_val - min_val)

            

# ACO

In [55]:
"""Ant System Metaherustic"""

import multiprocessing


class ACOptimizer(BaseLogger):
    
    def __init__(self, 
                 set_cover: SetCovering,
                 ants: int = 10,
                 evaporation_rate: float = 0.0,
                 alpha: float = 1.0,
                 beta:float = 0.0) -> None:
        
        """
        ACO algorithm initialization
        
        Args:
            saet_cover (SetCovering): set cover class with the subsets and costs
            ants (int): Number of ants in the instance
            evaporation_date (float):  pheromone's evaporation speed for each iteration
            
        """
        super().__init__()
        self.set_cover = set_cover
        self.ants = ants
        self.evaporation_rate = evaporation_rate
        self.heuristic_alpha = alpha
        self.heuristic_beta = beta
        self.set_initial_pheromones = self.__calc_pheromone()
        #self.set_pheromones = [ 1 / self.set_cover.total_subsets for pheromone in self.set_initial_pheromones]
        self.set_pheromones = self.set_initial_pheromones
        self.initialized = False
        
    def __str__(self):
        return "ACO(ants = {ants}, evaporation_rate = {evaporation_rate})".format(ants = self.ants, evaporation_rate = self.evaporation_rate)
    
    
    def __calc_obj_function(self):
        pass
    
    def __calc_pheromone(self):
        
        """Calculates pheromone level by each set based on the length of the set
        
        """
        
        pheromones = []
        for i, subset in enumerate(self.set_cover.subsets):
                
            length_score =  len(subset) / self.set_cover.max_len_set
            rarity_score =  self.set_cover.set_probabilities[i]
            cost_score   =  self.set_cover.costs[i]
                
            metaheuristic_information = length_score * 0.1 + rarity_score * 0.6 + (1 / cost_score) * 0.3
            pheromones.append(metaheuristic_information)
            
        return pheromones
            
    
    def __initialize(self):
        
        pool = ACOptimizer.make_pool()
        result = pool.map(self.set_cover.cover, [ ant for ant in range(self.ants) ])
        pool.close()
        return result
    
    def __update_probabilities(self, solutions) -> None:
        """Update probabilities based on past solutions and evaporation rate"""
        
        self.set_pheromones = [pheromone * (1 - self.evaporation_rate)  for pheromone in self.set_pheromones]
        
        for solution in solutions:
            for i, subset in enumerate(solution[0]):
                self.set_pheromones[i] += self.set_initial_pheromones[i]
        
        
    def build_ant_solution(self, ant):
        
        #pool = ACOptimizer.make_pool()
        solutions = self.__set_cover_ant_solution()
        #pool.close()
        return solutions
    
    def __set_cover_ant_solution(self):
        
        probabilities = self.__weight_pheromones(self.set_pheromones)
        all_sets = [*range(self.set_cover.total_subsets)]
        
        covered = set()
        selected_subsets = []
        cost = 0
        while covered != self.set_cover.universe:

            subset_idx = np.random.choice(a = all_sets , p = probabilities)
            subset     = set( self.set_cover.subsets[subset_idx] )

            if subset_idx in selected_subsets:
                continue

            selected_subsets.append(subset_idx)
            covered |= subset

            cost += self.set_cover.costs[subset_idx]
    
        
        self.logger.info(f">>> Total covering cost: {cost}")
        self.logger.info(f">>> Total subsets selected: {len(selected_subsets)}")
        
        return selected_subsets, cost
    
    def __weight_pheromones(self, pheromones):
        total_sum     = sum(pheromones)
        return [ val / total_sum  for val in pheromones ]
    
    def __select_top_n_sets(self, arr, n=100):
        return np.argsort(arr)[-n:]
    
    
    def local_search(self, solution, dropout = 0.015):
        
        best_sets = self.__select_top_n_sets(self.set_pheromones)
        sample_size = int( (1 - dropout) * len(solution[0]) )
        
        dropout_solution = random.sample(solution[0], sample_size)
        for bst in best_sets:
            if bst not in dropout_solution:
                dropout_solution.append(bst)
                
        covered = set(dropout_solution)
        cost = sum([ self.set_cover.costs[subset_idx] for subset_idx in dropout_solution])
        complement_sets = [subset for subset in range(self.set_cover.total_subsets) if subset not in dropout_solution]
        while covered != self.set_cover.universe:
            
            subset_idx = np.random.choice(a = complement_sets)
            subset     = set( self.set_cover.subsets[subset_idx] )
            
            if subset_idx in dropout_solution:
                continue
                
            dropout_solution.append(subset_idx)
            covered |= subset

            cost += self.set_cover.costs[subset_idx]
        
        self.logger.info(f">>> ------------------------------------------------------------------------------")
        self.logger.info(f">>> LOCAL SEARCH: Total covering cost: {cost}")
        self.logger.info(f">>> LOCAL SEARCH: Total subsets selected: {len(selected_subsets)}")
        
        return dropout_solution, cost
        
        
    @staticmethod
    def make_pool():
        
        AVAILABLE_CORES = multiprocessing.cpu_count()
        pool = multiprocessing.Pool(AVAILABLE_CORES)
        
        return pool
        
    
    def execute(self, iterations=2, mode='min', early_stopping_count=20, verbose=True) :
        
        
        # 1. Inicialization
        # Calculate pheromones, 
        f_solutions = self.__initialize()
        self.__update_probabilities(f_solutions)
        for _ in range(iterations):
            print(f" iteratcio {_}: ")
            pool = ACOptimizer.make_pool()
            solutions = pool.map(self.build_ant_solution, [1 for i in range(self.ants)])
            
            # local search
            ls_solutions = pool.map(self.local_search, solutions)
            iteration_solutions = [ls_solution if ls_solution[1] < solution else solution for ls_solution, solution in zip(ls_solutions, solutions)]
            pool.close() 
            # Global update pheromones
            self.__update_probabilities(iteration_solutions)
            
        pool.close() 
        return solutions
        
        



## Set Covering Pipeline

In [26]:
a = [1,2,3]
b = [4,5,6]

for a,b in zip(a,b):
    print(a,b)

1 4
2 5
3 6


In [56]:
set_cover_data = DataReader(file = "../aco_sc_data/sample_500_aco.txt")
set_cover_instance = SetCovering(set_cover_data.sets, costs=set_cover_data.costs)
aco_optimizer = ACOptimizer(set_cover=set_cover_instance, ants = 3)

In [57]:
ant_solutions = aco_optimizer.execute(iterations=3)

Model Logger - INFO - [2021-11-28 22:58:07,434] - >>> Total covering cost: 5600
Model Logger - INFO - [2021-11-28 22:58:07,434] - >>> Total covering cost: 5600
Model Logger - INFO - [2021-11-28 22:58:07,439] - >>> Total subsets selected: 2850
Model Logger - INFO - [2021-11-28 22:58:07,439] - >>> Total subsets selected: 2850
Model Logger - INFO - [2021-11-28 22:58:07,456] - >>> Total covering cost: 5622
Model Logger - INFO - [2021-11-28 22:58:07,456] - >>> Total covering cost: 5622
Model Logger - INFO - [2021-11-28 22:58:07,463] - >>> Total subsets selected: 2859
Model Logger - INFO - [2021-11-28 22:58:07,463] - >>> Total subsets selected: 2859
Model Logger - INFO - [2021-11-28 22:58:07,466] - >>> Total covering cost: 5353
Model Logger - INFO - [2021-11-28 22:58:07,466] - >>> Total covering cost: 5353
Model Logger - INFO - [2021-11-28 22:58:07,475] - >>> Total subsets selected: 2725
Model Logger - INFO - [2021-11-28 22:58:07,475] - >>> Total subsets selected: 2725


 iteratcio 0: 


Model Logger - INFO - [2021-11-28 22:58:09,902] - >>> Total covering cost: 5552
Model Logger - INFO - [2021-11-28 22:58:09,902] - >>> Total covering cost: 5552
Model Logger - INFO - [2021-11-28 22:58:09,929] - >>> Total subsets selected: 2828
Model Logger - INFO - [2021-11-28 22:58:09,929] - >>> Total subsets selected: 2828
Model Logger - INFO - [2021-11-28 22:58:09,946] - >>> Total covering cost: 5552
Model Logger - INFO - [2021-11-28 22:58:09,946] - >>> Total covering cost: 5552
Model Logger - INFO - [2021-11-28 22:58:09,961] - >>> Total subsets selected: 2828
Model Logger - INFO - [2021-11-28 22:58:09,958] - >>> Total covering cost: 5552
Model Logger - INFO - [2021-11-28 22:58:09,961] - >>> Total subsets selected: 2828
Model Logger - INFO - [2021-11-28 22:58:09,958] - >>> Total covering cost: 5552
Model Logger - INFO - [2021-11-28 22:58:09,966] - >>> Total subsets selected: 2828
Model Logger - INFO - [2021-11-28 22:58:09,966] - >>> Total subsets selected: 2828


KeyboardInterrupt: 

Process ForkPoolWorker-179:
Process ForkPoolWorker-177:
Process ForkPoolWorker-178:
Process ForkPoolWorker-182:
Process ForkPoolWorker-181:
Process ForkPoolWorker-180:
Process ForkPoolWorker-183:
Process ForkPoolWorker-184:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abdala9512/.pyenv/versions/3.9.7/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/abdala9512/.pyenv/versions/3.9.7/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/abdala9512/.pyenv/versions/3.9.7/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/abdala9512/.pyenv/versions/3.9.7/lib/python3.9/multiprocessing/process.py", line 315, in _boots

In [24]:
random.sample([1,2,3], 2)

[2, 1]

# Results reporting and analytics

In [ ]:
from collections import Counter

In [ ]:
len(set_cover_data.sets)

In [ ]:
b_set = set(tuple(x) for x in set_cover_data.sets)
b = [ list(x) for x in b_set ]
len(b)

In [13]:
aco_optimizer

In [ ]:
test = [4,1,2,502]
[item_final_scores[i] for i in test]

In [ ]:
set_cover_instance = SetCovering(set_cover_data.sets, costs=set_cover_data.costs)
min(set_cover_instance.set_scores)

In [ ]:
ant_sol = set_cover_instance.cover()

In [4]:
test_logger = BaseLogger()

In [6]:
test_logger.logger.handlers

[<FileHandler /home/abdala9512/ant-colony-optimization-implementation/notebooks/app.log (NOTSET)>,
 <StreamHandler stderr (NOTSET)>]

In [5]:
np.argsort([10,8,2,5,67])

array([2, 3, 1, 0, 4])

In [1]:
import numpy  as np